In [ ]:
#Data transformation is done for train test split

In [8]:
import os

In [9]:
%pwd

'c:\\Users\\saket\\Downloads\\ML_end_to_end\\End_to_End_ML_project\\research'

In [10]:
os.chdir("../")

In [11]:
%pwd

'c:\\Users\\saket\\Downloads\\ML_end_to_end\\End_to_End_ML_project'

In [1]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [2]:
from mlproject.constants import *
from mlproject.utils.common import read_yaml, create_directories

In [4]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
        )

        return data_transformation_config

In [5]:
import os
from mlproject import logger
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

In [6]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config


    
    ## Note: You can add different data transformation techniques such as Scaler, PCA and all
    #You can perform all kinds of EDA in ML cycle here before passing this data to the model

    # I am only adding train_test_spliting cz this data is already cleaned up

    def dropcolumn(self):
        data=pd.read_csv(self.config.data_path)
        data=data.drop("Serial No.", axis=1)

        return data

    def MinMaxScaler(self):
        data = self.dropcolumn()
        scaler = MinMaxScaler(feature_range=(0, 1))
        data["GRE Score"] = scaler.fit_transform(data[["GRE Score"]])
        data["TOEFL Score"] = scaler.fit_transform(data[["TOEFL Score"]])

        return data

    def train_test_spliting(self):

        data=self.MinMaxScaler()
        # Split the data into training and test sets. (0.75, 0.25) split.
        train, test = train_test_split(data)

        train.to_csv(os.path.join(self.config.root_dir, "train.csv"),index = False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"),index = False)

        logger.info("Splited data into training and test sets")
        logger.info(train.shape)
        logger.info(test.shape)

        print(train.shape)
        print(test.shape)

In [13]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.train_test_spliting()
except Exception as e:
    raise e

[2025-01-08 21:47:19,006: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-01-08 21:47:19,006: INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-08 21:47:19,006: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-01-08 21:47:19,006: INFO: common: created directory at: artifacts]
[2025-01-08 21:47:19,006: INFO: common: created directory at: artifacts/data_transformation]
[2025-01-08 21:47:19,023: INFO: 56740654: Splited data into training and test sets]
[2025-01-08 21:47:19,025: INFO: 56740654: (300, 8)]
[2025-01-08 21:47:19,025: INFO: 56740654: (100, 8)]
(300, 8)
(100, 8)
